**prerpocess data into pkl**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os
py_file_location = "/content/drive/Proyek Mandiri"

sys.path.append(os.path.abspath(py_file_location))

In [ ]:
#install Magnitude Medium GloVe for pre trained word embedding
!pip install pymagnitude


In [ ]:
import spacy
import argparse
from tqdm import tqdm
import pandas as pd
import pickle
import json
from functools import partial
from collections import Counter


In [ ]:
from tools import counter_to_distribution

In [ ]:
spacy_en = spacy.load("en")

In [ ]:
create_examples_dict = {
    "DD": create_examples_DD
}

In [ ]:
#Function for tokenized text data
def clean(text, max_sequence_length):
    """
        text: a piece of text in str
        max_sequence_length: the max sequence length for each utterance
        return: a list tokenized cleaned words
    """
    # lower case
    text = text.lower()
    
    # other cleaning processes
    
    # tokenization
    return [token.text for token in spacy_en.tokenizer(text)][:max_sequence_length]


In [ ]:
#function for create examples, i.e (uttarance, speaker, emotion, mask)
def clean_examples(examples, max_sequence_length):
    """
        examples: a list of examples, each example is a list of (utterance, speaker, emotion, mask)
        max_sequence_length: the max sequence length for each utterance
        return: a list tokenized cleaned examples
    """
    cleaned_examples = []
    for ex in tqdm(examples):
        cleaned_examples.append([(clean(utterance, max_sequence_length), speaker, emotion, mask) 
                                 for utterance, speaker, emotion, mask in ex])
    return cleaned_examples

**Mulai pre proses data**

Test dataset

In [ ]:
#for data test
with open("/content/drive/MyDrive/Proyek Mandiri/data/test_dialogues.txt".format(split), "r") as f:
        conversations = f.readlines()
with open("/content/drive/MyDrive/Proyek Mandiri/data/test_dialogues_emotion.txt".format(split), "r") as f:
        emotions = f.readlines()
print("{0} split has {1} conversations".format(split, len(conversations)))
    
examples = []
max_conv_length = max([len(conv.split("__eou__")[:-1]) for conv in conversations])
print("max_conv_length: ", max_conv_length)
    
dummy_utterance = "this is a dummy sentence"
dummy_emotion = emotions[0].strip().split(" ")[0]
dummy_speaker = "Speaker A"
i=0
for conv, emo in zip(conversations, emotions):
        utterances = [utter.strip() for utter in conv.split("__eou__")][:-1]
        i=i+1
        # add speaker info
        speakers = []
        for idx, utter in enumerate(utterances):
            if idx%2 == 0:
                speaker = "Speaker A"
            else:
                speaker = "Speaker B"
            speakers.append(speaker)
        
        # add emotion
        if (i==1000):
            utter_emotions = [emotion.strip() for emotion in emo.split(" ")]
        else :
            utter_emotions = [emotion.strip() for emotion in emo.split(" ")][:-1]
        assert len(utterances) == len(speakers) == len(utter_emotions)
                
        # pad dummy utterances
        conv_length = len(utterances)
        masks = conv_length*[1] + (max_conv_length-conv_length)*[0]
        utterances += (max_conv_length-conv_length)*[dummy_utterance]
        utter_emotions += (max_conv_length-conv_length)*[dummy_emotion]
        speakers += (max_conv_length-conv_length)*[dummy_speaker]
        
        # create examples
        examples.append(list(zip(utterances, speakers, utter_emotions, masks)))

In [ ]:
print(examples[:5])

[[('Hey man , you wanna buy some weed ?', 'Speaker A', '0', 1), ('Some what ?', 'Speaker B', '6', 1), ('Weed ! You know ? Pot , Ganja , Mary Jane some chronic !', 'Speaker A', '0', 1), ('Oh , umm , no thanks .', 'Speaker B', '0', 1), ('I also have blow if you prefer to do a few lines .', 'Speaker A', '0', 1), ('No , I am ok , really .', 'Speaker B', '0', 1), ('Come on man ! I even got dope and acid ! Try some !', 'Speaker A', '0', 1), ('Do you really have all of these drugs ? Where do you get them from ?', 'Speaker B', '0', 1), ('I got my connections ! Just tell me what you want and I ’ ll even give you one ounce for free .', 'Speaker A', '0', 1), ('Sounds good ! Let ’ s see , I want .', 'Speaker B', '0', 1), ('Yeah ?', 'Speaker A', '3', 1), ('I want you to put your hands behind your head ! You are under arrest !', 'Speaker B', '0', 1), ('this is a dummy sentence', 'Speaker A', '0', 0), ('this is a dummy sentence', 'Speaker A', '0', 0), ('this is a dummy sentence', 'Speaker A', '0', 0)

In [ ]:
 examples = clean_examples(examples, max_sequence_length)

In [ ]:
#saving data test (pkl form) into drive
path_to_save = "/content/drive/MyDrive/Proyek Mandiri/data/DD/test.pkl".format(dataset, split)
print("Saving data to {0}".format(path_to_save))
with open(path_to_save, "wb") as f:
    pickle.dump(examples, f)

Saving data to /content/drive/MyDrive/Proyek Mandiri/data/DD/test.pkl


Pre process for Train data set  

In [ ]:
with open("/content/drive/MyDrive/Proyek Mandiri/data/train_dialogues.txt".format(train), "r") as f:
        conversations = f.readlines()

In [ ]:
with open("/content/drive/MyDrive/Proyek Mandiri/data/train_dialogues_emotion.txt".format(split), "r") as f:
        emotions = f.readlines()

In [ ]:
  print("{0} split has {1} conversations".format(split, len(conversations)))

train split has 11118 conversations


In [ ]:
examples = []
max_conv_length = max([len(conv.split("__eou__")[:-1]) for conv in conversations])
print("max_conv_length: ", max_conv_length)

In [ ]:
dummy_utterance = "this is a dummy sentence"
dummy_emotion = emotions[0].strip().split(" ")[0]
dummy_speaker = "Speaker A"
for conv, emo in zip(conversations, emotions):
        utterances = [utter.strip() for utter in conv.split("__eou__")][:-1]
        
        # add speaker info
        speakers = []
        for idx, utter in enumerate(utterances):
            if idx%2 == 0:
                speaker = "Speaker A"
            else:
                speaker = "Speaker B"
            speakers.append(speaker)
        
        # add emotion
        utter_emotions = [emotion.strip() for emotion in emo.split(" ")[:-1]]
        assert len(utterances) == len(speakers) == len(utter_emotions)
                
        # pad dummy utterances
        conv_length = len(utterances)
        masks = conv_length*[1] + (max_conv_length-conv_length)*[0]
        utterances += (max_conv_length-conv_length)*[dummy_utterance]
        utter_emotions += (max_conv_length-conv_length)*[dummy_emotion]
        speakers += (max_conv_length-conv_length)*[dummy_speaker]
        
        # create examples
        examples.append(list(zip(utterances, speakers, utter_emotions, masks)))

In [ ]:
#print mylist[:50]
print(examples[:5])

[[('Say , Jim , how about going for a few beers after dinner ?', 'Speaker A', '0', 1), ('You know that is tempting but is really not good for our fitness .', 'Speaker B', '0', 1), ('What do you mean ? It will help us to relax .', 'Speaker A', '0', 1), ("Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?", 'Speaker B', '0', 1), ("I guess you are right.But what shall we do ? I don't feel like sitting at home .", 'Speaker A', '0', 1), ('I suggest a walk over to the gym where we can play singsong and meet some of our friends .', 'Speaker B', '0', 1), ("That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .", 'Speaker A', '4', 1), ('Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .', 'Speaker B', '4', 1), ("Good.Let ' s go now .", 'Speaker A', '4', 1), ('All right .', 'Speaker B', '4', 1), ('this is a dummy s

In [ ]:
max_sequence_length = 30

In [ ]:
examples = clean_examples(examples, max_sequence_length)

In [ ]:
#saving data test (pkl form) into drive
path_to_save = "/content/drive/MyDrive/Proyek Mandiri/data/DD/train.pkl".format(dataset, split)
print("Saving data to {0}".format(path_to_save))
with open(path_to_save, "wb") as f:
    pickle.dump(examples, f)

Pre proses for val dataset

In [ ]:
with open("/content/drive/MyDrive/Proyek Mandiri/data/val_dialogues.txt".format(train), "r") as f:
        conversations = f.readlines()

In [ ]:
with open("/content/drive/MyDrive/Proyek Mandiri/data/val_dialogues_emotion.txt".format(split), "r") as f:
        emotions = f.readlines()

In [ ]:
examples = []
max_conv_length = max([len(conv.split("__eou__")[:-1]) for conv in conversations])
print("max_conv_length: ", max_conv_length)

In [ ]:
dummy_utterance = "this is a dummy sentence"
dummy_emotion = emotions[0].strip().split(" ")[0]
dummy_speaker = "Speaker A"
for conv, emo in zip(conversations, emotions):
        utterances = [utter.strip() for utter in conv.split("__eou__")][:-1]
        
        # add speaker info
        speakers = []
        for idx, utter in enumerate(utterances):
            if idx%2 == 0:
                speaker = "Speaker A"
            else:
                speaker = "Speaker B"
            speakers.append(speaker)
        
        # add emotion
        utter_emotions = [emotion.strip() for emotion in emo.split(" ")[:-1]]
        assert len(utterances) == len(speakers) == len(utter_emotions)
                
        # pad dummy utterances
        conv_length = len(utterances)
        masks = conv_length*[1] + (max_conv_length-conv_length)*[0]
        utterances += (max_conv_length-conv_length)*[dummy_utterance]
        utter_emotions += (max_conv_length-conv_length)*[dummy_emotion]
        speakers += (max_conv_length-conv_length)*[dummy_speaker]
        
        # create examples
        examples.append(list(zip(utterances, speakers, utter_emotions, masks)))

In [ ]:
examples = clean_examples(examples, max_sequence_length)

In [ ]:
#saving data val (pkl form) into drive
path_to_save = "/content/drive/MyDrive/Proyek Mandiri/data/DD/val.pkl".format(dataset, split)
print("Saving data to {0}".format(path_to_save))
with open(path_to_save, "wb") as f:
    pickle.dump(examples, f)